# The `config.yaml`

The tools and recipes included in the [Clarity/Cadenza](https://github.com/claritychallenge/clarity) repository have been designed to be integrated into Python scripts that entrants may use or adapt according to their needs. However, for convenience, the baseline Cadenza tools can also be accessed via the command line interface (CLI) through shell scripts.

The Python and shell scripts included in the repository make use of [hydra](https://hydra.cc/) and [Submitit](https://hydra.cc/docs/plugins/submitit_launcher/), two technologies which streamline the configuration and parallel operation of python code on both local and high-performance computing (HPC) environments.

The use of Hydra for configuration allows the existing shell scripts to be easily redirected to include new audio data and modify the various parameters of the recipe.

## Inspecting Existing Configuration

All baselines take configurable variables from yaml files in the same directory as the shell script. In Cadenza, these files are typically named as `config.yaml`. However, other names may be used if more than one config file is needed.

Let's inspect the contents of a typical config file. The parameters may change depending on the challenge. However, all config files will have a `path` section, information about the `sample rates`, amplification parameters, evaluation parameters, hydra dependant parameters and others depending on te challenge.

```yaml
# Path Section
path:
  root: ../../cadenza_data_demo/cad1/task1
  metadata_dir: ${path.root}/metadata
  music_dir: ${path.root}/audio/musdb18hq
  music_file: ${path.metadata_dir}/musdb18.valid.json
  listeners_file: ${path.metadata_dir}/listeners.valid.json
  music_segments_test_file: ${path.metadata_dir}/musdb18.segments.test.json
  exp_folder: ./exp_${separator.model} # folder to store enhanced signals and final results


# Sample rates:
sample_rate: 44100       # sample rate of the input mixture
stem_sample_rate: 24000  # sample rate output stems
remix_sample_rate: 32000 # sample rate for output remixed signal

# Amplification:
nalr:
  nfir: 220
  sample_rate: ${sample_rate}

apply_compressor: False
compressor:
  threshold: 0.35
  attenuation: 0.1
  attack: 50
  release: 1000
  rms_buffer_size: 0.064

# Evaluation
evaluate:
  set_random_seed: True
  small_test: False
  batch_size: 1  # Number of batches
  batch: 0       # Batch number to evaluate


# Others
team_id: T001

soft_clip: True

separator:
  model: demucs   # demucs or openunmix
  device: ~

# hydra config
hydra:
  run:
    dir: ${path.exp_folder}
```

## Overriding a hydra config file

The general form for overriding a parameter in the `CLI` is dot indexed. For the following entry in a `config.yaml` file:

```yaml
A:
  B:
    parameter_0: some_value
    parameter_1: some_other_value
```

The `CLI` syntax to override those values would be:

```bash
user:~$ python myPythonScript.py A.B.parameter_0="new_value" A.B.parameter_1="another_new_value"
```

In the Clarity/Cadenza baselines, the enhancement and evaluation scripts share the same config file, which simplifies changes in the configuration. These parameters may include:

* Paths for the locations of audio files, metadata and the saving location for the generated files
* Parameters for the hearing aid amplification: NAL-R or Multiband Dynamic Range Compressor
* Parameters for the challenge evaluator
* Parameters for Hydra to run

One parameter that must be always be set is `path.root`. This parameter points to where the dataset is located in your system. This is the only parameter that must be set for the scripts to run. All other parameters have default values. 

e.g.,

```bash
user:~$ python myPythonScript.py path.root='/path/to/root/of/dataset'
```

## Shell Scripts

Typically, as stated above, all the work is done within python with configurable variables supplied by a `config.yaml` file, which is parsed by Hydra inside the python code.

The execution of this code is performed in the CLI and new configuration variable values are supplied as arguments to override defaults.